In [ ]:
# Find items starting with a publication or experiment set
from dcicutils import ff_utils
from functions.notebook_functions import *
import time
time1 = time.time()
env = 'data'
my_auth = get_key('koray_data')   #use your key name in keypairs.json
schema_name = get_schema_names(my_auth) 

sets = ['c23f2ffe-144a-437c-9e36-3b799529e6bb']  # place the uuid of the item here
store, uuids = record_object_es(sets, my_auth, schema_name, add_pc_wfr=False)
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print(time2-time1)

In [ ]:
#### This part should only run once!

transfer_env = 'fourfront-mastertest'
transfer_key = ff_utils.get_authentication_with_server({}, ff_env=transfer_env)
# reverse lookup dictionary for schema names

rev_schema_name = {}
for key, name in schema_name.items():
    rev_schema_name[name] = schema_name[key]

my_types = [i for i in ORDER if i in store.keys()]

second_round_items = {}

# Round I - only put the required - skip if exists already
for a_type in my_types:
    print(a_type)
    obj_type = rev_schema_name[a_type]
    # find required field
    schema_info = ff_utils.get_metadata('/profiles/{}.json'.format(a_type), key=transfer_key)
    req_fields = schema_info['required']
    ids = schema_info['identifyingProperties']
    first_fields = list(set(req_fields+ids))
    remove_existing_items = []
    counter=0
    print(len(store[a_type]), 'items exist on source')
    for an_item in store[a_type]:
        counter += 1
        #print(counter, an_item['uuid'])
        # does the item exist
        exists = False
        try:
            existing = ff_utils.get_metadata(an_item['uuid'], key=transfer_key)
            exists = True
        except:
            exists = False
        # skip the items that exists
        if exists and existing:
            remove_existing_items.append(an_item['uuid'])
            print("{} {} can not post item".format(obj_type, an_item['uuid']))
            continue
 
        post_first = {key:value for (key,value) in an_item.items() if key in first_fields}
        ff_utils.post_metadata(post_first, obj_type, key = transfer_key)
   
    second_round_items[a_type] = [i for i in store[a_type] if i['uuid'] not in remove_existing_items]
    print(len(second_round_items[a_type]), 'items transfered to target')
    print()

    

In [ ]:
# Round II - patch the rest
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    if not second_round_items[a_type]:
        continue 
    for an_item in second_round_items[a_type]:
        counter += 1
        #print(counter, an_item['uuid'])

        ##TEMP DISABLE ATTACHMENTS
#         if 'attachment' in an_item.keys():
#             del an_item['attachment']

        ff_utils.patch_metadata(an_item, obj_id = an_item['uuid'], key = transfer_key)
        


In [ ]:
move_files = False

import boto3
s3 = boto3.resource('s3')

#source_addresses
source_health = ff_utils.get_metadata('/health', key = my_auth)
source_raw = source_health['file_upload_bucket'] 
source_pf = source_health['processed_file_bucket'] 
source_att = source_health['blob_bucket']

#target_addresses
target_health = ff_utils.get_metadata('/health', key = transfer_key)
target_raw = target_health['file_upload_bucket'] 
target_pf = target_health['processed_file_bucket'] 
target_att = target_health['blob_bucket'] 

# Round III - move attachments
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    for an_item in second_round_items[a_type]:
        if 'attachment' in an_item.keys():
            at_key = an_item['attachment']['blob_id']
            copy_source = {'Bucket': source_att, 'Key': at_key}
            s3.meta.client.copy(copy_source, target_att, at_key)
            print('attachment copied')


In [ ]:
# Round IV - move files
for a_type in my_types:
    if a_type in ['file_processed']:
        source_file_bucket = source_pf
        target_file_bucket = target_pf
    elif a_type in ['file_reference', 'file_fastq', 'file_microscopy', 'file_fasta', 'file_calibration']:
        source_file_bucket = source_raw
        target_file_bucket = target_raw
    else:
        continue
        
    for an_item in second_round_items[a_type]:
        file_key = ff_utils.get_metadata(an_item['uuid'], key = my_auth)['upload_key']
        copy_source = {'Bucket': source_file_bucket, 'Key': file_key}
        try:
            s3.meta.client.copy(copy_source, target_file_bucket, file_key)
        except Exception as e:
            print(an_item['uuid'], 'can not be copied')
            raise e
        print('file copied')
